In [1]:
from my_packages import *
from appgeopy import *

In [3]:
shapefile_fld = "prediction_output/"
output_shp = glob(os.path.join(shapefile_fld, "*.shp"))
output_shp[:3]

['prediction_output\\gtwr_prediction_chunk001.shp',
 'prediction_output\\gtwr_prediction_chunk002.shp',
 'prediction_output\\gtwr_prediction_chunk003.shp']

In [20]:
combined_gdf = pd.DataFrame(data=None, index=None)

# select_shp = output_shp[0]
for select_shp in tqdm(output_shp):
    gdf = gpd.read_file(select_shp)
    
    new_cols = ["intercept", "CUMDISP_coeff", "predicted", "pred_var", "pred_stdev", "time", "geometry"]
    
    gdf = gdf.rename({old_col:new_col for old_col, new_col in zip(gdf.columns, new_cols)}, axis=1)
    
    x_twd97 = gdf.geometry.x
    y_twd97 = gdf.geometry.y
    
    pointkey = [f"X{int(x*1000)}Y{int(y*1000)}" for x,y in zip(x_twd97, y_twd97)]
    
    gdf.insert(loc=0, column="PointKey", value=pointkey)

    combined_gdf = pd.concat([combined_gdf, gdf], ignore_index=True)

  0%|          | 0/120 [00:00<?, ?it/s]

In [24]:
combined_gdf = combined_gdf.sort_values(by=["time", "PointKey"])
combined_gdf.to_pickle(r"Layer_1~CUMDISP_gaussian_b17_lambda0d5.xz")